# Reconocimiento facial

##### El objetivo de este proyecto es desarrollar un sistema de reconocimiento facial capaz de identificar a cinco personas diferentes utilizando tres algoritmos de reconocimiento facial:

- EigenFace
- FisherFace 
- LBPH (Local Binary Patterns Histograms).

##### El sistema utilizará imágenes de entrenamiento de las cinco personas objetivo para construir modelos de reconocimiento facial con cada algoritmo. Una vez entrenados, estos modelos serán capaces de reconocer y etiquetar las caras de las cinco personas en tiempo real.

##### Importar librerias.

In [2]:
import numpy as np 
import cv2 as cv
import os

faces = cv.CascadeClassifier(cv.data.haarcascades +'haarcascade_frontalface_alt.xml')

#### Función para recortar las fotografías a una dimensión uniforme. 

In [2]:
def escala(imx, escala):
    width = int(imx.shape[1] * escala / 100)
    height = int(imx.shape[0] * escala / 100)
    size = (width, height)
    im = cv.resize(imx, size, interpolation = cv.INTER_AREA)
    return im

##### El código anterior es un código que ayuda a realizar la recopilación de las fotos de los rostros. Este código fue ejecutado en 5 ocasiones para lograr capturar la información necesaria para dedicar a los modelos. 

In [ ]:
cap = cv.VideoCapture(0)
i = 0 

while True:
    ret, frame = cap.read()
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    rostros = rostro.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in rostros:
        cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cara_recortada = frame[y:y+h, x:x+w]
        cara_recortada = cv.resize(cara_recortada, (100, 100), interpolation=cv.INTER_AREA)
        cv.imwrite('D:/IA/caras/brenda/img' + str(i) + '.png', cara_recortada)

    cv.imshow('rostros', frame)
    i += 1
    #print(i)

    k = cv.waitKey(1)
    if k == 27:
        break

cap.release()
cv.destroyAllWindows()

#### El siguiente código se recopila las imágenes y las etiquetas con el objetivo de crear el dataset con el cual serán educados los modelos. 

In [3]:
dataSet = 'D:/IA/caras'
faces  = os.listdir(dataSet)
print(faces)

['rodrigo', 'abimael', 'dante', 'leo', 'brenda']


## Modelo 1 "EigenFace"

In [3]:
dataSet = 'D:/IA/caras'
faces  = os.listdir(dataSet)
print(faces)

labels = []
facesData = []
label = 0 
for face in faces:
    facePath = dataSet+'/'+face
    for faceName in os.listdir(facePath):
        labels.append(label)
        facesData.append(cv.imread(facePath+'/'+faceName,0))
    label = label + 1
print(np.count_nonzero(np.array(labels)==0)) 

faceRecognizer = cv.face.EigenFaceRecognizer_create()
faceRecognizer.train(facesData, np.array(labels))
faceRecognizer.write('face.xml')


['rodrigo', 'abimael', 'dante', 'leo', 'brenda']
983


In [ ]:
faceRecognizer = cv.face.EigenFaceRecognizer_create()
faceRecognizer.read('face.xml')

cap = cv.VideoCapture(0)
rostro = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_alt.xml')

while True:
    ret, frame = cap.read()
    if not ret:break

    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    cpGray = gray.copy()

    rostros = rostro.detectMultiScale(gray, 1.3, 3)

    for (x, y, w, h) in rostros:
        frame2 = cpGray[y:y + h, x:x + w]
        frame2 = cv.resize(frame2, (100, 100), interpolation=cv.INTER_CUBIC)
        result = faceRecognizer.predict(frame2)

        if result[1] > 5700: #3800
            cv.putText(frame, '{}'.format(faces[result[0]]), (x, y - 25), 2, 1.1, (0, 255, 0), 1, cv.LINE_AA)
            cv.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        else:
            cv.putText(frame, 'Desconocido', (x, y - 20), 2, 0.8, (0, 0, 255), 1, cv.LINE_AA)
            cv.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

    cv.imshow('frame', frame)
    k = cv.waitKey(1)
    if k == 27:
        break

cap.release()
cv.destroyAllWindows()



## Modelo 2 "FisherFace"

In [9]:
import time

In [6]:
dataSet = 'D:/IA/caras'
faces  = os.listdir(dataSet)
print(faces)

labels = []
facesData = []
label = 0 
for face in faces:
    facePath = dataSet+'/'+face
    for faceName in os.listdir(facePath):
        labels.append(label)
        facesData.append(cv.imread(facePath+'/'+faceName,0))
    label = label + 1
print(np.count_nonzero(np.array(labels)==0)) 

FisherFace = cv.face.FisherFaceRecognizer_create()
start_time = time.time()
FisherFace.train(facesData, np.array(labels))
training_time = time.time() - start_time
FisherFace.write('face2.xml')

['rodrigo', 'abimael', 'dante', 'leo', 'brenda']
983


In [7]:
print(f'Tiempo de entrenamiento modelo "FisherFace" : {training_time} segundos')

Tiempo de entrenamiento modelo "FisherFace" : 5431.443983078003 segundos


In [6]:
FisherFace = cv.face.FisherFaceRecognizer_create()
FisherFace.read('face2.xml')

cap = cv.VideoCapture(0)
rostro = cv.CascadeClassifier(cv.data.haarcascades +'haarcascade_frontalface_alt.xml')
while True:
    ret, frame = cap.read()
    if ret == False: break
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    cpGray = gray.copy()
    rostros = rostro.detectMultiScale(gray, 1.3, 3)
    for(x, y, w, h) in rostros:
        frame2 = cpGray[y:y+h, x:x+w]
        frame2 = cv.resize(frame2,  (100,100), interpolation=cv.INTER_CUBIC)
        result = FisherFace.predict(frame2)
        if result[1] > 500:
            cv.putText(frame,'{}'.format(faces[result[0]]),(x,y-25),2,1.1,(0,255,0),1,cv.LINE_AA)
            cv.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
        else:
            cv.putText(frame,'Desconocido',(x,y-20),2,0.8,(0,0,255),1,cv.LINE_AA)
            cv.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)
    cv.imshow('frame', frame)
    k = cv.waitKey(1)
    if k == 27:
        break
cap.release()
cv.destroyAllWindows()


## Modelo 3 "LBPH"

In [10]:
dataSet = 'D:/IA/caras'
faces  = os.listdir(dataSet)
print(faces)

labels = []
facesData = []
label = 0 
for face in faces:
    facePath = dataSet+'/'+face
    for faceName in os.listdir(facePath):
        labels.append(label)
        facesData.append(cv.imread(facePath+'/'+faceName,0))
    label = label + 1
print(np.count_nonzero(np.array(labels)==0)) 

LBPHFace = cv.face.LBPHFaceRecognizer_create()
start_time = time.time()
LBPHFace.train(facesData, np.array(labels))
training_time = time.time() - start_time
LBPHFace.write('face3.xml')

['rodrigo', 'abimael', 'dante', 'leo', 'brenda']
983


In [11]:
print(f'Tiempo de entrenamiento modelo "LBPH" : {training_time} segundos')

Tiempo de entrenamiento modelo "LBPH" : 4.7530837059021 segundos


In [13]:
LBPHFace = cv.face.LBPHFaceRecognizer_create()
LBPHFace.read('face3.xml')

cap = cv.VideoCapture(0)
rostro = cv.CascadeClassifier(cv.data.haarcascades +'haarcascade_frontalface_alt.xml')
while True:
    ret, frame = cap.read()
    if ret == False: break
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    cpGray = gray.copy()
    rostros = rostro.detectMultiScale(gray, 1.3, 3)
    for(x, y, w, h) in rostros:
        frame2 = cpGray[y:y+h, x:x+w]
        frame2 = cv.resize(frame2,  (100,100), interpolation=cv.INTER_CUBIC)
        result = FisherFace.predict(frame2)
        if result[1] > 70:
            cv.putText(frame,'{}'.format(faces[result[0]]),(x,y-25),2,1.1,(0,255,0),1,cv.LINE_AA)
            cv.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
        else:
            cv.putText(frame,'Desconocido',(x,y-20),2,0.8,(0,0,255),1,cv.LINE_AA)
            cv.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)
    cv.imshow('frame', frame)
    k = cv.waitKey(1)
    if k == 27:
        break
cap.release()
cv.destroyAllWindows()